In [11]:
import chromadb
chroma_client = chromadb.Client()

In [12]:
collection = chroma_client.create_collection(name="resume_data", )


UniqueConstraintError: Collection resume_data already exists

In [13]:
# Read and add documents to ChromaDB
import os

# Example: reading text files from a directory
documents = []
metadatas = []
ids = []

doc_dir = "docs"
for i, filename in enumerate(os.listdir(doc_dir)):
    if filename.endswith(".txt"):  # or any other extension you're using
        with open(os.path.join(doc_dir, filename), 'r') as file:
            content = file.read()
            documents.append(content)
            metadatas.append({"source": filename})
            ids.append(f"doc_{i}")

# Add documents to the collection
collection.add(
    documents=documents,
    metadatas=metadatas,
    ids=ids
)

Insert of existing embedding ID: doc_0
Add of existing embedding ID: doc_0


In [14]:
import keys
OPENAI_API_KEY = OPENAI_API_KEY

In [15]:
from openai import OpenAI
client = OpenAI(api_key = OPENAI_API_KEY)

completion = client.chat.completions.create(
  model="gpt-4o",
  messages=[
    {"role": "developer", "content": "You are an assistant who talks about Abhinav Duvvuri's resume. NEVER ANSWER ANYTHING OUTSIDE THE CONTEXT. NEVER EVER"},
    {"role": "user", "content": "Hello!"}
  ],
  stream=True
)

for chunk in completion:
  print(chunk.choices[0].delta.content, end='', flush=True)

Hello! How can I assist you with information about Abhinav Duvvuri's resume today?None

In [16]:
def get_relevant_context(query, k=3):
    results = collection.query(
        query_texts=[query],
        n_results=k
    )
    # Combine the retrieved documents into a single context string
    context = "\n\n".join(results['documents'][0])
    return context

# Example chat with context
query = "Tell me about Abhinav's experience"
context = get_relevant_context(query)


Number of requested results 3 is greater than number of elements in index 1, updating n_results = 1


In [17]:
while True:
    # Example chat with context
    query = input("What do you want to know about Abhinav: ")
    context = get_relevant_context(query)
    if query.lower() == "bye":
        print('Have a lovely day!')
        break
    completion = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": "You are an assistant who talks about Abhinav Duvvuri's resume. Only use the provided context to answer questions. If you can't find the information in the context, say you don't have that information."},
            {"role": "system", "content": f"Context from resume: {context}"},
            {"role": "user", "content": query}
        ],
        stream=True
    )

    for chunk in completion:
        print(chunk.choices[0].delta.content, end='', flush=True)

Number of requested results 3 is greater than number of elements in index 1, updating n_results = 1


Abhinav is skilled with the following tech stack:

- Tools: Object Oriented (OOP) Python, SQL, Semantic Kernel, C#, Azure, Azure AI, Service-Now, Kubernetes, Docker
- Experience: RAG implementations, GenAI Agentic Frameworks, Fast API, scalable REST APIs, LLM prompt Engineering, performance tuningNone

Number of requested results 3 is greater than number of elements in index 1, updating n_results = 1


Hello! If you have any questions about Abhinav Duvvuri's resume, feel free to ask, and I'll do my best to provide the information based on the context given.None

Number of requested results 3 is greater than number of elements in index 1, updating n_results = 1


How can I assist you with Abhinav Duvvuri's resume?None

Number of requested results 3 is greater than number of elements in index 1, updating n_results = 1


How can I assist you regarding Abhinav Duvvuri's resume?None

In [9]:
import base64
import vertexai
from vertexai.preview.generative_models import GenerativeModel, SafetySetting, Part, Tool
from vertexai.preview.generative_models import grounding


def multiturn_generate_content():
    vertexai.init(project="concise-option-448310-n8", location="us-central1")
    tools = [
        Tool.from_retrieval(
            retrieval=grounding.Retrieval(
                source=grounding.VertexAISearch(datastore="projects/concise-option-448310-n8/locations/global/collections/default_collection/dataStores/resume-portfolio_1737284302495"),
            )
        ),
    ]
    model = GenerativeModel(
        "gemini-1.5-flash-002",
        tools=tools,
        system_instruction=[textsi_1]
    )
    chat = model.start_chat()
    print(chat.send_message(
        ["""hi"""],
        generation_config=generation_config,
        safety_settings=safety_settings
    ))
    print(chat.send_message(
        ["""what is abhinav good at?"""],
        generation_config=generation_config,
        safety_settings=safety_settings
    ))

textsi_1 = """You are an AI bot with context of my resume. You answer questions about me from my resume. You do not use any context outside that and say that you cant answer questions whenever you don't know."""

generation_config = {
    "max_output_tokens": 8192,
    "temperature": 0.3,
    "top_p": 0.95,
}

safety_settings = [
    SafetySetting(
        category=SafetySetting.HarmCategory.HARM_CATEGORY_HATE_SPEECH,
        threshold=SafetySetting.HarmBlockThreshold.OFF
    ),
    SafetySetting(
        category=SafetySetting.HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT,
        threshold=SafetySetting.HarmBlockThreshold.OFF
    ),
    SafetySetting(
        category=SafetySetting.HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT,
        threshold=SafetySetting.HarmBlockThreshold.OFF
    ),
    SafetySetting(
        category=SafetySetting.HarmCategory.HARM_CATEGORY_HARASSMENT,
        threshold=SafetySetting.HarmBlockThreshold.OFF
    ),
]

multiturn_generate_content()

candidates {
  content {
    role: "model"
    parts {
      text: "Hello!  I\'m ready to answer your questions based on your resume. Please ask away.\n"
    }
  }
  finish_reason: STOP
  grounding_metadata {
    retrieval_metadata {
    }
  }
  avg_logprobs: -0.058454297837756929
}
usage_metadata {
  prompt_token_count: 43
  candidates_token_count: 21
  total_token_count: 64
}
model_version: "gemini-1.5-flash-002"

candidates {
  content {
    role: "model"
    parts {
      text: "Based on the provided resume, Abhinav Duvvuri excels in several areas:\n\n* **AI Engineering and Development:** He has extensive experience in developing and implementing enterprise-level AI solutions, including RAG implementations, GenAI Agentic Frameworks, and LLM prompt engineering.  He\'s proficient in using various tools and technologies like Azure AI, OpenAI APIs, Semantic Kernel, and MongoDB.  His projects demonstrate expertise in building AI copilots,  creating reusable AI libraries, and integrating